In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import glob

client_credentials_manager = SpotifyClientCredentials(
        "26399552a8ce4d1285397254189cac50",
        "fdacfffa2dd34dbeb127dedb459f7ea3")
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [2]:
def getTrackFeatures(trackURI):
    ger_data = pd.DataFrame()
    for track in trackURI:
        audio = sp.audio_analysis(track)
        segments_data =  pd.DataFrame(audio["segments"])
        pitch = segments_data.pitches.apply(pd.Series)
        pitch.columns = ["p" + str(i) for i in range(1,13)]
        timbre = segments_data.timbre.apply(pd.Series)
        timbre.columns = ["t" + str(i) for i in range(1,13)]
        segments_data = segments_data.drop(["pitches", "timbre", "loudness_end"],
                                           axis = 1)
        segments_data = segments_data.join([pitch, timbre])
        segments_data["track_id"] = track
        ger_data = ger_data.append(segments_data)
    return ger_data

In [ ]:
Africa = ['DZ', 'EG', 'MA', 'ZA', 'TN']
Asia = ['BH', 'HK', 'IN', 'ID', 'IL', 'JP', 'JO', 'KW', 
        'LB', 'MY', 'OM', 'PS', 'PH', 'QA', 'SA', 'SG', 
        'TW', 'TH', 'AE', 'VN']
Europe = ['AD', 'AT', 'BE', 'BG', 'CY', 'CZ', 'DK', 'EE', 
          'FI', 'FR', 'DE', 'GR', 'HU', 'IS', 'IE', 'IT', 
          'LV', 'LI', 'LT', 'LU', 'MT', 'MC', 'NL', 'NO', 
          'PL', 'PT', 'RO', 'SK', 'ES', 'SE', 'CH', 'TR', 'GB']
NorthAmerica = ['CA', 'CR', 'DO', 'SV', 'GT', 'HN', 'MX', 'NI', 'PA', 'US']
SouthAmerica = ['AR', 'BO', 'BR', 'CL', 'CO', 'EC', 'PY', 'PE', 'UY']
Oceania = ['AU', 'NZ']

countriesOfInterest = Africa + Asia + Oceania + SouthAmerica
print(len(countriesOfInterest))
for year in range(2019,2000, -1):
    tmstamp = str(year) + "-06-15T12:00:00"
    for countryCode in countriesOfInterest:
        if len(glob.glob("Raw Playlist Data\\" + countryCode + "_" + str(year) + ".csv")) == 0:
            country = sp.featured_playlists(country = countryCode, limit = 50)
            plist = country["playlists"]["items"]

            plist = pd.DataFrame.from_dict(plist)
        
            print(countryCode, "has", len(plist), "playlists in ", str(year))
            plist.to_csv("Raw Playlist Data\\" + countryCode + "_" + str(year) + ".csv", index = False)
            for uri in plist["uri"]:
                currplist = sp.playlist(uri)
                trackURI = []
                for track in currplist["tracks"]["items"]:
                    try:
                        trackURI = trackURI + [track["track"]["uri"]]
                    except:
                        continue
                print("Getting track features for", uri, "in", countryCode)
                try:
                    trackData = getTrackFeatures(trackURI)
                    trackData.to_csv("Raw Track Data\\" + countryCode + "_" + str(year) + "_" + uri[17:] + ".csv", index = False)
                except:
                    continue
            

36
BH has 17 playlists in  2015
Getting track features for spotify:playlist:37i9dQZF1DX51I48C1U653 in BH
Getting track features for spotify:playlist:37i9dQZF1DX5cO1uP1XC1g in BH
Getting track features for spotify:playlist:37i9dQZF1DWYHBENPjyDpc in BH
Getting track features for spotify:playlist:37i9dQZF1DX1fzTwl8xjJf in BH
Getting track features for spotify:playlist:37i9dQZF1DX4rmnjMBxfCv in BH
Getting track features for spotify:playlist:37i9dQZF1DXdcjKKBPqh1t in BH
Getting track features for spotify:playlist:37i9dQZF1DWWhuJcTXO5hW in BH
Getting track features for spotify:playlist:37i9dQZF1DXd3AhRYJnfcl in BH
Getting track features for spotify:playlist:37i9dQZF1DX5lckay3OqOG in BH
Getting track features for spotify:playlist:37i9dQZF1DWYKzhEkKdUkK in BH
Getting track features for spotify:playlist:37i9dQZF1DWU486KSiznWZ in BH
Getting track features for spotify:playlist:37i9dQZF1DX2TRYkJECvfC in BH
Getting track features for spotify:playlist:37i9dQZF1DXaHuo0R6jTxZ in BH
Getting track featu

Getting track features for spotify:playlist:37i9dQZF1DWUa8ZRTfalHk in MY
Getting track features for spotify:playlist:37i9dQZF1DXayb37p8Vuer in MY
Getting track features for spotify:playlist:37i9dQZF1DWXLeA8Omikj7 in MY
Getting track features for spotify:playlist:37i9dQZF1DWXML2LendtLJ in MY
Getting track features for spotify:playlist:37i9dQZF1DX4SBhb3fqCJd in MY
Getting track features for spotify:playlist:37i9dQZF1DXdd3gw5QVjt9 in MY
Getting track features for spotify:playlist:37i9dQZF1DWSJHnPb1f0X3 in MY
OM has 17 playlists in  2015
Getting track features for spotify:playlist:37i9dQZF1DX51I48C1U653 in OM
Getting track features for spotify:playlist:37i9dQZF1DX5cO1uP1XC1g in OM
Getting track features for spotify:playlist:37i9dQZF1DWYHBENPjyDpc in OM
Getting track features for spotify:playlist:37i9dQZF1DX1fzTwl8xjJf in OM
Getting track features for spotify:playlist:37i9dQZF1DX4rmnjMBxfCv in OM
Getting track features for spotify:playlist:37i9dQZF1DXdcjKKBPqh1t in OM
Getting track features

Getting track features for spotify:playlist:37i9dQZF1DX7zXqDWRZPD1 in TW
Getting track features for spotify:playlist:37i9dQZF1DX42o4SsMqqwn in TW
Getting track features for spotify:playlist:37i9dQZF1DXdd3gw5QVjt9 in TW
Getting track features for spotify:playlist:37i9dQZF1DX6uIc2aogDNQ in TW
Getting track features for spotify:playlist:37i9dQZF1DX71PXzGB2fd7 in TW
Getting track features for spotify:playlist:37i9dQZF1DX6mXxqnWLIPb in TW
Getting track features for spotify:playlist:37i9dQZF1DX1uG5byNIgDA in TW
TH has 15 playlists in  2015
Getting track features for spotify:playlist:37i9dQZF1DX2ZbLEg4W5iS in TH
Getting track features for spotify:playlist:37i9dQZF1DXc51TI5dx7RC in TH
Getting track features for spotify:playlist:37i9dQZF1DX8peoGqrSg0H in TH
Getting track features for spotify:playlist:37i9dQZF1DX9tPFwDMOaN1 in TH
Getting track features for spotify:playlist:37i9dQZF1DX72Zxs6RduBK in TH
Getting track features for spotify:playlist:37i9dQZF1DWVMjQ4irK5mi in TH
Getting track features

Getting track features for spotify:playlist:37i9dQZF1DXdqrBLlPKz6L in PE
Getting track features for spotify:playlist:37i9dQZF1DX9C8KzGEUKV4 in PE
Getting track features for spotify:playlist:37i9dQZF1DX57NRq0pf7ea in PE
Getting track features for spotify:playlist:37i9dQZF1DX8sljIJzI0oo in PE
Getting track features for spotify:playlist:37i9dQZF1DX2piJKuRdKIA in PE
Getting track features for spotify:playlist:37i9dQZF1DWVhcOv4ieuap in PE
Getting track features for spotify:playlist:37i9dQZF1DWU8yQ1nTMCfh in PE
Getting track features for spotify:playlist:37i9dQZF1DX9c9fpd0lxJ5 in PE
Getting track features for spotify:playlist:37i9dQZF1DX4ICTqceOnSJ in PE
Getting track features for spotify:playlist:37i9dQZF1DX1UHxedJfnRM in PE
Getting track features for spotify:playlist:37i9dQZF1DX9tPFwDMOaN1 in PE
Getting track features for spotify:playlist:37i9dQZF1DX2xhsrO0GW1M in PE
Getting track features for spotify:playlist:37i9dQZF1DWYzTSrc7SiSZ in PE
Getting track features for spotify:playlist:37i9dQZ

Getting track features for spotify:playlist:37i9dQZF1DXa2EiKmMLhFD in ID
Getting track features for spotify:playlist:37i9dQZF1DWY8Ogms1X39p in ID
Getting track features for spotify:playlist:37i9dQZF1DWTbDegdS0BGI in ID
Getting track features for spotify:playlist:37i9dQZF1DWYJVaIADJesD in ID
Getting track features for spotify:playlist:37i9dQZF1DX7i7SKKuAK4o in ID
Getting track features for spotify:playlist:37i9dQZF1DWTRkBYeInhLG in ID
IL has 12 playlists in  2014
Getting track features for spotify:playlist:37i9dQZF1DX1mdpXpgpDiF in IL
Getting track features for spotify:playlist:37i9dQZF1DX8dEi3l8JSdP in IL
Getting track features for spotify:playlist:37i9dQZF1DWVbXsIvZBlOB in IL
Getting track features for spotify:playlist:37i9dQZF1DX9sLipKPkV9T in IL
Getting track features for spotify:playlist:37i9dQZF1DXcXpkHQIToU1 in IL
Getting track features for spotify:playlist:37i9dQZF1DWSYF6geMtQMW in IL
Getting track features for spotify:playlist:37i9dQZF1DWYbUY40ZDAwb in IL
Getting track features

Getting track features for spotify:playlist:37i9dQZF1DXaHuo0R6jTxZ in PS
Getting track features for spotify:playlist:37i9dQZF1DX4rmnjMBxfCv in PS
Getting track features for spotify:playlist:37i9dQZF1DWYHBENPjyDpc in PS
Getting track features for spotify:playlist:37i9dQZF1DXd3AhRYJnfcl in PS
Getting track features for spotify:playlist:37i9dQZF1DX7Gwi28HhdBP in PS
Getting track features for spotify:playlist:37i9dQZF1DXcC6YLqj8ET8 in PS
Getting track features for spotify:playlist:37i9dQZF1DWWhuJcTXO5hW in PS
Getting track features for spotify:playlist:37i9dQZF1DWUQM3rmTXpBR in PS
Getting track features for spotify:playlist:37i9dQZF1DX68x6hand0cN in PS
PH has 9 playlists in  2014
Getting track features for spotify:playlist:37i9dQZF1DWVjrTQlvqTHF in PH
Getting track features for spotify:playlist:37i9dQZF1DXcZQSjptOQtk in PH
Getting track features for spotify:playlist:37i9dQZF1DWX9VXBLRgDqu in PH
Getting track features for spotify:playlist:37i9dQZF1DX53n2vsU9kOm in PH
Getting track features 

Getting track features for spotify:playlist:37i9dQZF1DX7Gwi28HhdBP in AE
Getting track features for spotify:playlist:37i9dQZF1DX4rmnjMBxfCv in AE
Getting track features for spotify:playlist:37i9dQZF1DX0ysO3y1BFXN in AE
Getting track features for spotify:playlist:37i9dQZF1DWYKzhEkKdUkK in AE
Getting track features for spotify:playlist:37i9dQZF1DWWhuJcTXO5hW in AE
Getting track features for spotify:playlist:37i9dQZF1DXcC6YLqj8ET8 in AE
Getting track features for spotify:playlist:37i9dQZF1DXe3aCmUoBd8n in AE
VN has 12 playlists in  2014
Getting track features for spotify:playlist:37i9dQZF1DX5G3iiHaIzdf in VN
Getting track features for spotify:playlist:37i9dQZF1DXbYM3nMM0oPk in VN
Getting track features for spotify:playlist:37i9dQZF1DWWdlSG3xntgM in VN
Getting track features for spotify:playlist:37i9dQZF1DXbSWYCNwaARB in VN
Getting track features for spotify:playlist:37i9dQZF1DX1e2VSJFudND in VN
Getting track features for spotify:playlist:37i9dQZF1DX6sCJFCJIfiB in VN
Getting track features

In [ ]:
glob.glob("Raw Playlist Data\\" + countryCode + "_" + str(year) + ".csv")

In [ ]:
for year in range(2019,2000, -1):
    print(year)

In [ ]:
glob.glob("Raw Playlist Data\\" + countryCode + "_" + str(year) + ".csv")